# Flight Web Scrapping:

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import selenium.webdriver
from datetime import date
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from dateutil.relativedelta import relativedelta
import csv
import linecache
import sys

In [2]:
def PrintException():
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno, f.f_globals)
    print('EXCEPTION IN ({}, LINE {} "{}"): {}'.format(filename, lineno, line.strip(), exc_obj))

In [3]:
def scrap(origin,destin,trDate):
    global none
    baseDataUrl = "https://www.makemytrip.com/flight/search?itinerary="+ origin +"-"+ destin +"-"+ trDate +"&tripType=O&paxType=A-1_C-0_I-0&intl=false&cabinClass=E"

    try:
        driver = selenium.webdriver.Chrome(r"D:\Internship\Project 4\Car Price Prediction\chromedriver.exe") 
        print ("Requesting URL: " + baseDataUrl)

        driver.get(baseDataUrl)  
        print ("Webpage found ...")

        element_xpath = '//div[@id="left-side--wrapper"]'
        
        
        element = WebDriverWait(driver, 15).until(EC.visibility_of_element_located((By.XPATH, element_xpath)))

        print ("Scrolling document upto bottom ...")
        for j in range(1, 100):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        
        body = driver.find_element(By.XPATH,"/html/body/div[1]/div/div[2]/div[2]/div/div[2]/div[2]").get_attribute("innerHTML")

        print("Closing Chrome ...")
        driver.quit()

        print("Getting data from DOM ...")
        soupBody = BeautifulSoup(body)
        
        spanFlightName = soupBody.find_all("p", 'boldFont blackText airlineName')
        print("spanFlightName  = " + str(len(spanFlightName)))

        Time = soupBody.find_all("p", {"class":"appendBottom2 flightTimeInfo"})			# Tags with Departure Time
        print("Time Count in Scraping  = " + str(len(Time)))

        City = soupBody.find_all("font", {"color": "#000000"})
        print("City Count in Scraping  = " + str(len(City)))

        spanFlightCost = soupBody.find_all("p", {"class": "blackText fontSize18 blackFont white-space-no-wrap"})			# Tags with Flight Cost
        print("spanFlightCost Count in Scraping  = " + str(len(spanFlightCost)))

        pFlightStops = soupBody.find_all("p", {"class": "flightsLayoverInfo"})
        print("pFlightStops Count in Scraping  = " + str(len(pFlightStops)))
        
        transform(spanFlightName,trDate,City,Time,spanFlightCost,pFlightStops)



   
    except :
        PrintException()
        print(trDate," scraping complete")
        
        none =True 

In [4]:
dates=(pd.date_range(start='20/10/2022', end='30/10/2022'))
New_date=[]
for i,date in enumerate(dates):
    day=str(date.day).zfill(2)   
    month=str(date.month).zfill(2)
    year=str(date.year)
    date=day+'/'+month+'/'+year  
    New_date.append(date)
print(New_date)

['20/10/2022', '21/10/2022', '22/10/2022', '23/10/2022', '24/10/2022', '25/10/2022', '26/10/2022', '27/10/2022', '28/10/2022', '29/10/2022', '30/10/2022']


D:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3369: UserWarning: Parsing '20/10/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  exec(code_obj, self.user_global_ns, self.user_ns)
D:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3369: UserWarning: Parsing '30/10/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
#Lists to store scraped data
Airline=[]
Journey_date=[]
From=[]
To=[]
Price=[]
Dtime=[]
Atime=[]
Stops=[]

flight_data = pd.DataFrame(np.nan, index=[0], columns=['Airline', 'Journey_date','From','To','Dtime','Atime','Stops','Price'])
flight_data.to_csv('Flight.csv')
 
def transform(spanFlightName,trDate,City,Time,spanFlightCost,pFlightStops):
    
    Airline.clear()
    Journey_date.clear()
    Price.clear()
    Stops.clear()
    From.clear()
    Dtime.clear()
    To.clear()
    Atime.clear()
    
    global flight_data
    print("Transform Function Running.... ")
    for i in range(0,len(spanFlightName)):
        
        Airline.append(spanFlightName[i].string)
        Journey_date.append(trDate)
        Price.append(spanFlightCost[i].string)
        Stops.append(pFlightStops[i].string)
        
      
    for j in range(0,len(Time)):
        
        if(j%2==0):
            
            From.append(City[j].string)
            Dtime.append(Time[j].string)
        else:
            
            To.append(City[j].string)
            Atime.append(Time[j].string) 

            
            
    temp = {'Airline':Airline,
            'Journey_date':Journey_date,
            'From':From,
            'To':To,
            'Dtime':Dtime,
            'Atime':Atime,
            'Stops':Stops,
            'Price':Price}
    df=pd.DataFrame(temp)
    df.to_csv('Flight.csv', mode='a', header=False)
    flight_data = pd.concat([flight_data,df])
    flight_data=flight_data.reset_index(drop=True)
    display(flight_data)
    print('Lenght : ',len(flight_data))

In [ ]:
list_of_dest=['GOI','AMD','TIR','BLR','BOM','DEL']
for i in range(0,len(list_of_dest)):
    orgin=list_of_dest[i]
    for_to=list_of_dest
    print("--------------")
    for j in range(0,len(for_to)):
        destin=for_to[j]
        none = False
        print("origin = "+orgin)
        print("destin = "+destin+'\n')
        if orgin!=destin:
            for i,date in enumerate(New_date) :
                if none !=True:
                    scrap(orgin,destin,date)
                    print(" ")
                else:
                    break
            else: 
                print( "-----Skiped-----")
                print(" ")

C:\Users\HP\AppData\Local\Temp\ipykernel_14688\3653695753.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = selenium.webdriver.Chrome(r"D:\Internship\Project 4\Car Price Prediction\chromedriver.exe")


--------------
origin = GOI
destin = GOI

origin = GOI
destin = AMD

Requesting URL: https://www.makemytrip.com/flight/search?itinerary=GOI-AMD-20/10/2022&tripType=O&paxType=A-1_C-0_I-0&intl=false&cabinClass=E
Webpage found ...
Scrolling document upto bottom ...
Closing Chrome ...
Getting data from DOM ...
spanFlightName  = 7
Time Count in Scraping  = 14
City Count in Scraping  = 17
spanFlightCost Count in Scraping  = 7
pFlightStops Count in Scraping  = 7
Transform Function Running.... 


,Airline,Journey_date,From,To,Dtime,Atime,Stops,Price
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Go First,20/10/2022,Get 100% Refund on Flight Cancellations,Goa,21:00,22:35,Non stop,"₹ 5,402"
2,IndiGo,20/10/2022,Ahmedabad,Goa,14:35,16:15,Non stop,"₹ 5,404"
3,SpiceJet,20/10/2022,Ahmedabad,Goa,07:50,09:30,Non stop,"₹ 4,526"
4,SpiceJet,20/10/2022,Ahmedabad,Goa,07:50,09:30,Non stop,"₹ 5,508"
5,SpiceJet,20/10/2022,Ahmedabad,Goa,21:05,22:55,Non stop,"₹ 5,508"
6,IndiGo,20/10/2022,Ahmedabad,MMTBFLEMI,23:05,None,Non stop,"₹ 5,509"
7,IndiGo,20/10/2022,HDFCEMI,Goa,06:45,13:55,1 stop via Mumbai,"₹ 7,399"


Lenght :  8
 


C:\Users\HP\AppData\Local\Temp\ipykernel_14688\3653695753.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = selenium.webdriver.Chrome(r"D:\Internship\Project 4\Car Price Prediction\chromedriver.exe")


Requesting URL: https://www.makemytrip.com/flight/search?itinerary=GOI-AMD-21/10/2022&tripType=O&paxType=A-1_C-0_I-0&intl=false&cabinClass=E
Webpage found ...
Scrolling document upto bottom ...
Closing Chrome ...
Getting data from DOM ...
spanFlightName  = 21
Time Count in Scraping  = 42
City Count in Scraping  = 45
spanFlightCost Count in Scraping  = 21
pFlightStops Count in Scraping  = 21
Transform Function Running.... 


,Airline,Journey_date,From,To,Dtime,Atime,Stops,Price
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Go First,20/10/2022,Get 100% Refund on Flight Cancellations,Goa,21:00,22:35,Non stop,"₹ 5,402"
2,IndiGo,20/10/2022,Ahmedabad,Goa,14:35,16:15,Non stop,"₹ 5,404"
3,SpiceJet,20/10/2022,Ahmedabad,Goa,07:50,09:30,Non stop,"₹ 4,526"
4,SpiceJet,20/10/2022,Ahmedabad,Goa,07:50,09:30,Non stop,"₹ 5,508"
5,SpiceJet,20/10/2022,Ahmedabad,Goa,21:05,22:55,Non stop,"₹ 5,508"
6,IndiGo,20/10/2022,Ahmedabad,MMTBFLEMI,23:05,None,Non stop,"₹ 5,509"
7,IndiGo,20/10/2022,HDFCEMI,Goa,06:45,13:55,1 stop via Mumbai,"₹ 7,399"
8,SpiceJet,21/10/2022,Get 100% Refund on Flight Cancellations,Goa,07:50,09:30,Non stop,"₹ 4,526"
9,SpiceJet,21/10/2022,Ahmedabad,Goa,21:05,22:55,Non stop,"₹ 5,403"


Lenght :  29
 


C:\Users\HP\AppData\Local\Temp\ipykernel_14688\3653695753.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = selenium.webdriver.Chrome(r"D:\Internship\Project 4\Car Price Prediction\chromedriver.exe")


Requesting URL: https://www.makemytrip.com/flight/search?itinerary=GOI-AMD-22/10/2022&tripType=O&paxType=A-1_C-0_I-0&intl=false&cabinClass=E


In [12]:
print(len(Airline),len(Journey_date),len(From),len(To),len(Dtime),len(Atime),len(Stops),len(Price))

76 76 76 76 76 76 76 76


In [13]:
print(Dtime)

['07:30', '06:30', '05:55', '02:00', '05:25', '06:30', '12:20', '07:20', '06:25', '07:15', '07:00', '07:55', '08:45', '12:05', '10:55', '08:00', '22:30', '07:00', '10:00', '08:50', '09:30', '09:55', '08:00', '17:55', '21:10', '06:00', '12:00', '05:15', '19:00', '20:00', '01:55', '10:20', '19:40', '06:10', '20:10', '15:45', '21:45', '23:00', '13:20', '21:20', '13:20', '14:15', '14:20', '16:25', '17:25', '18:30', '10:45', '19:10', '21:55', '13:40', '11:40', '05:40', '11:00', '19:55', '18:15', '09:00', '10:00', '19:00', '20:00', '22:30', '14:30', '12:50', '19:45', '07:00', '20:40', '16:40', '14:20', '16:45', '17:25', '06:25', '15:30', '17:45', '19:00', '14:00', '17:50', '13:15']


In [14]:
print(Atime)

['09:45', '23:15', '07:55', '04:15', '07:40', '08:45', '19:45', '09:35', '12:50', '09:40', '09:10', '10:05', '11:10', '17:45', '17:45', '10:15', None, '09:05', '12:15', '11:10', '11:35', '16:55', '10:10', '23:35', '23:25', '10:40', '14:20', '09:45', '21:10', '22:05', '08:20', '12:35', '21:50', '12:20', None, '18:00', None, None, '20:20', '23:35', '18:20', '16:30', '16:30', '18:40', '19:40', '20:50', None, None, '23:55', '19:45', '14:00', '10:10', '13:20', '22:15', '22:05', '11:15', '12:35', '21:15', '22:10', None, '18:15', '15:00', '21:55', '12:40', '22:50', '21:25', '16:30', '19:05', None, '17:00', '17:40', '20:05', '21:15', '16:15', '20:15', None]


In [15]:
#checking scraped Flight Price data
df=pd.read_csv("Flight.csv")
df

,Unnamed: 0,Airline,Journey_date,From,To,Dtime,Atime,Stops,Price
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,Go First,20/10/2022,Get 100% Refund on Flight Cancellations,Goa,21:00,22:35,Non stop,"₹ 5,402"
2,1,IndiGo,20/10/2022,Ahmedabad,Goa,14:35,16:15,Non stop,"₹ 5,404"
3,2,SpiceJet,20/10/2022,Ahmedabad,Goa,07:50,09:30,Non stop,"₹ 4,526"
4,3,SpiceJet,20/10/2022,Ahmedabad,Goa,07:50,09:30,Non stop,"₹ 5,508"
...,...,...,...,...,...,...,...,...,...
8715,71,Vistara,30/10/2022,New Delhi,Mumbai,17:45,20:05,Non stop,"₹ 26,851"
8716,72,Vistara,30/10/2022,New Delhi,Mumbai,19:00,21:15,Non stop,"₹ 26,851"
8717,73,Air India,30/10/2022,New Delhi,Mumbai,14:00,16:15,Non stop,"₹ 27,344"
8718,74,Air India,30/10/2022,New Delhi,Mumbai,17:50,20:15,Non stop,"₹ 27,344"


In [17]:
#saving Flight Price data as csv file
df.to_csv("flight_price.csv")